In [1]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np

In [16]:
!mkdir dataframe

# I- Data Pre-Processing

In [167]:
df = pd.read_csv("your_dataset.csv")

In [168]:
print(df.shape)
print(df.head())

(10000, 11)
         x1        x2          x3        x4        x5        x6        x7  \
0  1.205492  5.823226   98.837539 -1.075852  0.999205  0.911543  3.623558   
1  1.391530  3.611581   98.857197 -5.020318  0.677165  0.999492  3.413112   
2  1.692571 -0.887019  100.901276 -0.595548  0.177550 -0.915495  4.320264   
3  4.289320  1.416843  100.784735 -2.897154 -0.066972 -0.786173  2.093003   
4  0.542420 -1.010095  100.015580 -3.070705  0.088324 -0.242669  0.767942   

         x8        x9        x10  y  
0 -1.720267 -0.346191 -54.708330 -1  
1  4.253865  2.041603 -54.317291  1  
2  0.907834  3.126815 -56.397484 -1  
3  1.336237  2.183829 -56.197728  1  
4 -0.284683 -2.104145 -55.794045  1  


In [169]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.any():
        print("There are missing values in the dataframe.")
        print(missing_values)
    else:
        print("There are no missing values in the dataframe")

# Appel de la fonction
check_missing_values(df)

There are no missing values in the dataframe


In [144]:
# function that normalise the dataframe
def normalise_df(df):
    
    df_norm = (df - df.min()) / (df.max() - df.min())
    
    return df_norm

# function that splits the dataframe s.t : train set = 60% - val set = 20% - test set = 20%
def split_dataframe(df_x, df_y):
    
    split = int(df.shape[0] * 0.8)
    
    X_train = df_x.iloc[:split,:]
    Y_train = df_y.iloc[:split]

    X_test = df_x.iloc[split:,:]
    Y_test = df_y.iloc[split:]
    
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    Y_train = Y_train.to_numpy()
    Y_test = Y_test.to_numpy()
    
    return X_train, Y_train, X_test, Y_test


def save_set(X_train, Y_train, X_test, Y_test):
    np.save('X_train.npy', X_train)
    np.save('Y_train.npy', Y_train)
    np.save('X_test.npy', X_test)
    np.save('Y_test.npy', Y_test)
    

def pre_process_df(df):
    
    df = df.sample(frac=1).reset_index(drop=True) # mixed the dataframe
    
    X = df.iloc[:, :-1]
    Y = df.iloc[:, -1]
    
    X_norm = normalise_df(X)
    X_norm.insert(0,'bias',1)
    X_train, Y_train, X_test, Y_test = split_dataframe(X_norm, Y)
    save_set(X_train, Y_train, X_test, Y_test)
    
    return X_train, Y_train, X_test, Y_test

In [145]:
X_train, Y_train, X_test, Y_test = pre_process_df(df)

In [146]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(8000, 11)
(8000,)
(2000, 11)


# II- Perceptron Algorithm

In [3]:
def feature_expansion_2(X):
    
    X_exp = X.copy()
    
    # add square product
    for i in range(1, X.shape[1]):
        square_feature = (X[:, i] ** 2).reshape(X.shape[0], 1)
        X_exp = np.concatenate((X_exp, square_feature), axis=1)
    
    # add cross product
    for i in range(1, X.shape[1]):
        for j in range(i+1, X.shape[1]):
            cross_feature = (X[:, i] * X[:, j]).reshape(X.shape[0], 1)
            X_exp = np.concatenate((X_exp, cross_feature), axis=1)
    
    return X_exp

In [4]:
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')
X_train_exp = feature_expansion_2(X_train)
X_test_exp = feature_expansion_2(X_test)

In [7]:
class Perceptron:
    
    def __init__(self, feature_number=11):
        self.max_epoch = None
        self.feature_number = feature_number
        self.w = None
    
    def train(self, X_train, Y_train, param):
        
        self.w = np.zeros(self.feature_number)
        self.max_epoch = param

        for epoch in range(self.max_epoch):
            
            #we start an EPOCH
            update = False
            for i in range(X_train.shape[0]):

                dot_prod = Y_train[i] * np.dot(self.w, X_train[i,:])

                if (dot_prod <= 0):
                    self.w = self.w + Y_train[i] * X_train[i,:]
                    update = True

            if (update == False):
                break
        
        return self.w
        
    def predict(self, X_test):
        
        output = np.dot(X_test, self.w)
        Y_predicted = np.sign(output)
        
        return Y_predicted
    
    def evaluate(self, X_test, Y_test):
        
        Y_pred = self.predict(X_test)
        accuracy = np.mean(Y_pred == Y_test)
        
        return accuracy

In [59]:
def spliting_for_tuning_param(X_train, Y_train):
    split_idx = int(0.75 * X_train.shape[0])  # 75% for training, 25% for validation
    X_train_tuning = X_train[:split_idx, :]
    Y_train_tuning = Y_train[:split_idx]
    X_val_tuning = X_train[split_idx:, :]
    Y_val_tuning = Y_train[split_idx:]
    
    return X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning

In [60]:
def perceptron_tuning_parameter(p, list_param, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning):
    
    result = np.zeros(len(list_param))
    for i, param in enumerate(list_param):
        p.train(X_train_tuning, Y_train_tuning, param)
        result[i] = p.evaluate(X_val_tuning, Y_val_tuning)

    index_max = np.argmax(result)
    best_param = list_param[index_max]
    
    return best_param

In [61]:
X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning = spliting_for_tuning_param(X_train, Y_train)

In [62]:
X_train_tuning_exp, Y_train_tuning_exp, X_val_tuning_exp, Y_val_tuning_exp = spliting_for_tuning_param(X_train_exp, Y_train)

## a) Perceptron run without feature expansion

First we choose, among a define class, the best number of epoch to be carried out thanks to the function tuning_parameter

In [63]:
list_epoch_param = [i for i in range(1, 31)]
best_epoch_p = perceptron_tuning_parameter(Perceptron(11), list_epoch_param, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print(best_epoch_p)

1


Then we train the Perceptron using this number of epoch on the full train set

In [64]:
perceptron = Perceptron(11)
w_perceptron = perceptron.train(X_train, Y_train, best_epoch_p)
print('w =', w_perceptron)

w = [-3.          2.39717537  0.16794903 -1.31400493 -4.88529671  1.83689138
 -1.24603313  4.16327828  9.1343608   1.84258619 -1.37851589]


In [65]:
accuracy_train = perceptron.evaluate(X_train, Y_train)
accuracy_test = perceptron.evaluate(X_test, Y_test)
print('rate of good answer on train set : ', accuracy_train)
print('rate of good answer on test set : ', accuracy_test)

rate of good answer on train set :  0.715375
rate of good answer on test set :  0.7295


## b) Perceptron run with feature expansion of degree 2

We proceed in the same way when feature expansion. We first choose the best number of epoch to be carried out :

In [66]:
best_epoch_pfe = perceptron_tuning_parameter(Perceptron(66), list_epoch_param, X_train_tuning_exp, Y_train_tuning_exp, X_val_tuning_exp, Y_val_tuning_exp)
print(best_epoch_pfe)

27


And then we train our algorithm on the whole training set with feature expansion

In [67]:
perceptron_exp = Perceptron(66)
w_perceptron_exp = perceptron_exp.train(X_train_exp, Y_train, best_epoch_pfe)
print('w_exp =', w_perceptron_exp)

w_exp = [ 2.60000000e+01 -3.84548191e+01 -1.14367796e+02  2.45083154e+01
  3.79171999e+01 -2.70422837e+00  3.05210273e+00 -1.63408251e+01
  1.61352483e+01 -9.64159823e+01  4.86890852e+00 -1.63309062e+01
  2.43191669e+01  6.82633676e+00 -1.11674562e+01  1.80558865e+00
  9.91435304e+00 -1.53563786e+00  1.82868014e+01  2.49644178e+01
  1.32250308e+00  4.47711053e+01 -2.02581685e+01 -1.09123877e+01
 -5.81210337e+00 -4.61138611e+00  8.72077833e+00  1.15623519e+02
 -2.89902504e+00 -3.66745245e+00 -7.17865604e+01 -3.68475061e+00
 -7.62599642e+00 -2.84770090e+01  6.55563884e+00  1.64344156e+00
  2.79848543e+02 -3.12367611e+01  2.64820002e+01 -1.11209074e+00
  1.06881777e+01 -1.10957609e+00  1.26293747e+01 -5.39507104e+01
  1.29299122e+01 -1.83977756e-01  7.78282610e+00  1.18158638e+01
 -7.73495988e+01 -3.60187224e+01  1.07608557e+01 -7.42863388e-01
  4.80910988e+00  1.05283985e+01  6.34780135e+00  1.45115043e+00
 -1.56664123e+00  1.27887947e+01 -2.75053052e+01  3.62179041e+00
  8.71893923e+00 

In [68]:
accuracy_train_exp = perceptron_exp.evaluate(X_train_exp, Y_train)
accuracy_test_exp = perceptron_exp.evaluate(X_test_exp, Y_test)
print('rate of good answer on train set with expanded features : ', accuracy_train_exp)
print('rate of good answer on test set with expanded features : ', accuracy_test_exp)

rate of good answer on train set with expanded features :  0.892875
rate of good answer on test set with expanded features :  0.891


# II- SVM with Pegasos

In [29]:
class SVM:
    
    def __init__(self, feature_number=11):
        self.T = None
        self.lambda_param = None
        self.feature_number = feature_number
        self.w = None
    
    def train(self, X_train, Y_train, T, lambda_param):
        
        self.T = T                          # T is the number of rounds
        self.lambda_param = lambda_param    # lambda is the regularization coefficient
        
        sum_h = 0
        h = np.zeros(self.feature_number)
        sample_size = X_train.shape[0]
        
        for t in range(self.T):
            
            lr = (1 / (self.lambda_param * (t + 1)))
            i = np.random.randint(0, sample_size)
            condition = 1 - (Y_train[i] * np.dot(h, X_train[i,:]))
            
            if (condition > 0):
                grad_l = self.lambda_param * h + Y_train[i] * X_train[i,:]
            else:
                grad_l = self.lambda_param * h
                
            h = h - lr * grad_l
            sum_h = sum_h + h

        self.w = (sum_h / self.T)
        
        return self.w
        
    def predict(self, X_test):
        
        output = np.dot(X_test, self.w)
        Y_predicted = np.sign(output)
        
        return Y_predicted
    
    def evaluate(self, X_test, Y_test):
        
        Y_pred = self.predict(X_test)
        accuracy = np.mean(Y_pred == Y_test)
        
        return accuracy

In [90]:
def svm_tuning_parameter(algo, list_T, list_lr, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning):
    
    result = np.zeros((len(list_T), len(list_lr)))
    for l, t in enumerate(list_T):
        for i, lr in enumerate(list_lr):
            s = 0
            # we repeat it 3 time for each parameter combination in order to get a mean value since pegasos is stochastic
            for j in range(3):
                algo.train(X_train_tuning, Y_train_tuning, t, lr)
                s += algo.evaluate(X_val_tuning, Y_val_tuning)
                
            result[l,i] = s / 3   

    index_max = np.unravel_index(np.argmax(result), result.shape)
    best_T = list_T[index_max[0]]
    best_lr = list_lr[index_max[1]]
    
    return best_T, best_lr

## a) SVM run without feature expansion

In [77]:
list_T = [1000, 10000, 50000]
list_lr = [0.001, 0.01, 0.1, 1]
best_T, best_lr = svm_tuning_parameter(SVM(), list_T, list_lr, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best number of round T : ',best_T,'\n','best lambda : ', best_lr)

best number of round T :  10000 
 best learning rate :  0.01


In [30]:
svm = SVM()

In [78]:
w_svm = svm.train(X_train, Y_train, T=best_T, lambda_param=best_lr)
print('w =', w_svm)

w = [-49.54        -9.87690102 -25.70609082 -24.3749704  -37.34486616
 -27.84897922 -24.88441976 -26.61517831 -26.16444088 -27.79824458
 -24.70627206]


In [80]:
accuracy_svm_train = svm.evaluate(X_train, Y_train)
accuracy_svm = svm.evaluate(X_test, Y_test)
print('accuracy of svm on training set :', accuracy_svm_train)
print('accuracy of svm on test set :', accuracy_svm)

accuracy of svm on training set : 0.50125
accuracy of svm on test set : 0.499


## b) SVM run with feature expansion of degree 2

In [82]:
best_T_exp, best_lr_exp = svm_tuning_parameter(SVM(66), list_T, list_lr, X_train_tuning_exp, Y_train_tuning_exp, X_val_tuning_exp, Y_val_tuning_exp)
print('best number of round T feature expansion : ',best_T_exp,'\n','best lambda with feature expansion : ', best_lr_exp)

best number of round T feature expansion :  1000 
 best learning rate with feature expansion :  0.01


In [83]:
svm_exp = SVM(feature_number=66)

In [84]:
w_svm_exp = svm_exp.train(X_train_exp, Y_train, T=best_T_exp, lambda_param=best_lr_exp)
print('w =', w_svm_exp)

w = [-50.8        -10.62124289 -26.8768331  -24.98216769 -38.06728142
 -27.40050099 -25.47601963 -27.55281116 -26.89462393 -28.84960797
 -25.27840143  -3.44208536 -15.27046921 -16.1873455  -29.9857394
 -21.47845674 -19.57406723 -15.93746541 -15.130951   -17.56011336
 -16.72378416  -5.96725364  -5.17849847  -7.7570097   -5.32230392
  -5.37950729  -5.99454806  -5.95501127  -6.58288053  -5.31489141
 -13.33771468 -19.92264046 -14.24469498 -13.3053477  -14.84802982
 -14.4927237  -15.9462939  -13.24438382 -18.90836621 -13.54964132
  -7.42647957 -13.42474738 -13.12731242 -14.27210562  -8.48470345
 -21.00497157 -18.85231928 -20.54548836 -19.83117241 -21.5627088
 -18.75373673 -13.63955771 -14.66750776 -14.13721879 -15.14641257
 -13.60447174 -14.00676352 -13.59643968 -14.34516794 -17.93466622
 -14.85635585 -15.99478335 -13.85367328 -15.55759428 -13.46415964
 -14.26678144]


In [86]:
accuracy_svm_train_exp = svm_exp.evaluate(X_train_exp, Y_train)
accuracy_svm_exp = svm_exp.evaluate(X_test_exp, Y_test)
print('accuracy of svm on expanded training set :', accuracy_svm_train_exp)
print('accuracy of svm on expanded test set :', accuracy_svm_exp)

accuracy of svm on expanded training set : 0.50125
accuracy of svm on expanded test set : 0.499


# III- Regularized logistic classification

In [92]:
class Regularized_Logistic_Classification:
    
    def __init__(self, feature_number=11):
        self.T = None
        self.lambda_param = None
        self.feature_number = feature_number        
        self.w = None
    
    def train(self, X_train, Y_train, T, lambda_param):
        
        self.T = T                          # T is the number of rounds
        self.lambda_param = lambda_param    # lambda is the regularization coefficient
        sum_h = 0
        h = np.zeros(self.feature_number)
        sample_size = X_train.shape[0]
        
        for t in range(self.T):
            
            lr = (1 / (self.lambda_param * (t + 1)))
            i = np.random.randint(0, sample_size)
            
            num = Y_train[i] * X_train[i,:]
            den = np.log(2) * (1 + np.exp(Y_train[i] * np.dot(h, X_train[i,:])))
            
            grad_l = self.lambda_param * h - (num / den)
                
            h = h - lr * grad_l
            sum_h = sum_h + h

        self.w = (sum_h / self.T)
        
        return self.w
        
    def predict(self, X_test):
        
        output = np.dot(X_test, self.w)
        Y_predicted = np.sign(output)
        
        return Y_predicted
    
    def evaluate(self, X_test, Y_test):
        
        Y_pred = self.predict(X_test)
        accuracy = np.mean(Y_pred == Y_test)
        
        return accuracy

## a) Regularized logistic regression run without feature expansion

In [158]:
list_T = [1000, 10000, 50000]
list_lambda = [0.001, 0.01, 0.1, 1]
best_T_rlc, best_lambda_rlc = svm_tuning_parameter(Regularized_Logistic_Classification(), list_T, list_lambda, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best number of round T : ',best_T_rlc,'\n','best lambda : ', best_lambda_rlc)

/tmp/ipykernel_4068/1548082262.py:23: RuntimeWarning: overflow encountered in exp
  den = np.log(2) * (1 + np.exp(Y_train[i] * np.dot(h, X_train[i,:])))


best number of round T :  50000 
 best lambda :  0.001


In [159]:
rlc = Regularized_Logistic_Classification()

In [160]:
w_rlc = rlc.train(X_train, Y_train, best_T_rlc, best_lambda_rlc)
print('w_rlc =', w_rlc, '\n')
accuracy_train_rlc = rlc.evaluate(X_train, Y_train)
accuracy_test_rlc = rlc.evaluate(X_test, Y_test)
print('accuracy of rlc on training set :', accuracy_train_rlc)
print('accuracy of rlc on test set :', accuracy_test_rlc)

/tmp/ipykernel_4068/1548082262.py:23: RuntimeWarning: overflow encountered in exp
  den = np.log(2) * (1 + np.exp(Y_train[i] * np.dot(h, X_train[i,:])))


w_rlc = [-1.58410915  2.25098859 -0.96907334 -0.67188579 -2.49473774  0.79264228
 -0.85464782  2.33751037  5.56664856  0.85450092 -0.60236804] 

accuracy of rlc on training set : 0.71825
accuracy of rlc on test set : 0.7335


## b) Regularized logistic regression run with features expansion of degree 2

In [161]:
best_T_rlc_exp, best_lambda_rlc_exp = svm_tuning_parameter(Regularized_Logistic_Classification(66), list_T, list_lambda, X_train_tuning_exp, Y_train_tuning_exp, X_val_tuning_exp, Y_val_tuning_exp)
print('best number of round T with expansion feature for rlc: ', best_T_rlc_exp,'\n','best lambda with expansion feature for rlc : ', best_lambda_rlc_exp)

/tmp/ipykernel_4068/1548082262.py:23: RuntimeWarning: overflow encountered in exp
  den = np.log(2) * (1 + np.exp(Y_train[i] * np.dot(h, X_train[i,:])))


best number of round T with expansion feature for rlc:  50000 
 best lambda with expansion feature for rlc :  0.001


In [162]:
rlc_exp = Regularized_Logistic_Classification(feature_number=66)    

In [163]:
w_rlc_exp = rlc_exp.train(X_train_exp, Y_train, best_T_rlc_exp, best_lambda_rlc_exp)
accuracy_train_rlc_exp = rlc_exp.evaluate(X_train_exp, Y_train)
accuracy_test_rlc_exp = rlc_exp.evaluate(X_test_exp, Y_test)
print('accuracy of rlc on training set :', accuracy_train_rlc_exp)
print('accuracy of rlc on test set :', accuracy_test_rlc_exp)

/tmp/ipykernel_4068/1548082262.py:23: RuntimeWarning: overflow encountered in exp
  den = np.log(2) * (1 + np.exp(Y_train[i] * np.dot(h, X_train[i,:])))


accuracy of rlc on training set : 0.740125
accuracy of rlc on test set : 0.7545


# IV- Kernel Perceptron

In [116]:
def polynomial_kernel(X_1, X_2, degree):
        
    pol_K = (1 + np.matmul(X_1, X_2.T)) ** degree
    
    return pol_K

In [117]:
def gaussian_kernel(X_1, X_2, gamma):
    
    X1_norm = np.sum(X_1**2, axis=1).reshape(-1, 1) # create a (1, X_1.shape[0]) vector
    X2_norm = np.sum(X_2**2, axis=1).reshape(1, -1) # create a (X_2.shape[0], 1) vector
    
    # create a (X_1.shape[0], X_2.shape[0]) matrix where component (i, j) is ||X1_i - X2_j||² with X1_i the i (line) vector of X1
    X_norm = X1_norm + X2_norm - 2 * np.matmul(X_1, X_2.T) 
    
    gaussian_K = np.exp(-(0.5 * X_norm) / gamma)
    
    return gaussian_K

In [118]:
class Kernel_Perceptron:
    
    def __init__(self,  kernel):
        self.kernel = kernel
        self.max_epoch = None
        self.param_kernel = None
        self.w_index = None
        self.kernel_vectors = None
        self.kernel_vector_labels = None
    
    def train(self, X_train, Y_train, max_epoch, param_kernel):
        
        sample_size = X_train.shape[0]
        self.w_index = np.zeros(sample_size)
        self.kernel_vectors = X_train
        self.kernel_vector_labels = Y_train
        self.max_epoch = max_epoch
        self.param_kernel = param_kernel
        
        K = self.kernel(X_train, X_train, self.param_kernel)
        
        for epoch in range(self.max_epoch):
            
            update = False
            for i in range(sample_size):
                
                pred = np.sum(self.w_index * Y_train * K[:, i])
                
                if (np.sign(pred) != Y_train[i]):
                    self.w_index[i] += 1
                    update = True

            if (update == False):
                print('no update in last epoch')
                break
        
        
    def predict(self, X_test):
        
        K = self.kernel(X_test, self.kernel_vectors, self.param_kernel)
        Y_predicted = np.sign(np.sum(self.w_index * self.kernel_vector_labels * K, axis=1))

        return Y_predicted
    
    def evaluate(self, X_test, Y_test):
        
        Y_pred = self.predict(X_test)
        accuracy = np.mean(Y_pred == Y_test)
        
        return accuracy

In [165]:
def kernel_perceptron_tuning_parameter(algo, list_epoch, list_kernel_param, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning):
    
    result = np.zeros((len(list_epoch), len(list_kernel_param)))
    
    for l, e in enumerate(list_epoch):
        for i, param in enumerate(list_kernel_param):
            algo.train(X_train_tuning, Y_train_tuning, e, param)
            result[l,i] = algo.evaluate(X_val_tuning, Y_val_tuning)
        
        print(f'step {l+1} over {len(list_epoch)}')
            
    index_max = np.unravel_index(np.argmax(result), result.shape)
    best_epoch = list_epoch[index_max[0]]
    best_param = list_kernel_param[index_max[1]]
    
    return best_epoch, best_param

## a) Kernel Perceptron with polynomial kernel

#### Let's try some value for the polynome degree :

In [143]:
kernel_perceptron_pol = Kernel_Perceptron(polynomial_kernel)

##### With polynomial degree of 2

In [144]:
max_epoch = 10
polynomial_degree = 1
kernel_perceptron_pol.train(X_train, Y_train, max_epoch, polynomial_degree)
accuracy_pol_2 = kernel_perceptron_pol.evaluate(X_test, Y_test)
print(f'accuracy for Perceptron with polynomial kernel of degree {polynomial_degree} : ', accuracy_pol_2)

accuracy for Perceptron with polynomial kernel of degree 1 :  0.647


##### With polynomial degree of 3

In [145]:
polynomial_degree = 3
kernel_perceptron_pol.train(X_train, Y_train, max_epoch, polynomial_degree)
accuracy_pol_3 = kernel_perceptron_pol.evaluate(X_test, Y_test)
print(f'accuracy for Perceptron with polynomial kernel of degree {polynomial_degree} : ', accuracy_pol_3)

accuracy for Perceptron with polynomial kernel of degree 3 :  0.801


##### With polynomial degree of 6

In [146]:
polynomial_degree = 6
kernel_perceptron_pol.train(X_train, Y_train, max_epoch, polynomial_degree)
accuracy_pol_6 = kernel_perceptron_pol.evaluate(X_test, Y_test)
print(f'accuracy for Perceptron with polynomial kernel of degree {polynomial_degree} : ', accuracy_pol_6)

accuracy for Perceptron with polynomial kernel of degree 6 :  0.8965


#### Now, let's try to find the best hyperparameters :

In [128]:
list_epoch = [i for i in range(1, 51, 10)]
list_degree = [i for i in range(1, 16)]
best_epoch, best_degree = kernel_perceptron_tuning_parameter(Kernel_Perceptron(polynomial_kernel), list_epoch, list_degree, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best number of epoch : ',best_epoch,'\n','best polynomial degree : ', best_degree)

0
1
2
3
4
best number of epoch :  41 
 best polynomial degree :  14


In [151]:
kernel_perceptron_pol.train(X_train, Y_train, best_epoch, best_degree)
accuracy_train_pol = kernel_perceptron_pol.evaluate(X_train, Y_train)
accuracy_test_pol = kernel_perceptron_pol.evaluate(X_test, Y_test)
print(f'accuracy on train set for Perceptron with {best_epoch} epoch and polynomial kernel of degree {best_degree} : ', accuracy_train_pol)
print(f'accuracy on test set for Perceptron with {best_epoch} epoch and polynomial kernel of degree {best_degree} : ', accuracy_test_pol)

accuracy on train set for Perceptron with 41 epoch and polynomial kernel of degree 14 :  0.9325
accuracy on test set for Perceptron with 41 epoch and polynomial kernel of degree 14 :  0.922


## b) Kernel Perceptron with gaussian kernel

#### Let's try some value for the gamma parameter :

In [137]:
max_epoch = 10
kernel_perceptron_gauss = Kernel_Perceptron(gaussian_kernel)

##### with gamma = 1

In [138]:
gamma = 1
kernel_perceptron_gauss.train(X_train, Y_train, max_epoch, gamma)
accuracy_gauss_1 = kernel_perceptron_gauss.evaluate(X_test, Y_test)
print(f'accuracy for Perceptron with gaussian kernel with gamma {gamma} : ', accuracy_gauss_1)

accuracy for Perceptron with gaussian kernel with gamma 1 :  0.852


##### with gamma = 2

In [140]:
gamma = 2
kernel_perceptron_gauss.train(X_train, Y_train, max_epoch, gamma)
accuracy_gauss_2 = kernel_perceptron_gauss.evaluate(X_test, Y_test)
print(f'accuracy for Perceptron with gaussian kernel with gamma {gamma} : ', accuracy_gauss_2)

accuracy for Perceptron with gaussian kernel with gamma 2 :  0.9


#### Now, let's try to find the best hyperparameters :

In [141]:
list_gamma = [0.1, 0.5, 1, 2, 5, 7, 10]
best_epoch, best_gamma = kernel_perceptron_tuning_parameter(Kernel_Perceptron(gaussian_kernel), list_epoch, list_gamma, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best number of epoch : ',best_epoch,'\n','best gamma : ', best_gamma)

0
1
2
3
4
best number of epoch :  41 
 best gamma :  0.1


In [142]:
kernel_perceptron_gauss = Kernel_Perceptron(gaussian_kernel)
kernel_perceptron_gauss.train(X_train, Y_train, best_epoch, best_gamma)
accuracy_train_gauss = kernel_perceptron_gauss.evaluate(X_train, Y_train)
accuracy_test_gauss = kernel_perceptron_gauss.evaluate(X_test, Y_test)
print(f'accuracy on train set for Perceptron with {best_epoch} epoch and gaussian kernel with gamma {gamma} : ', accuracy_train_gauss)
print(f'accuracy on test set for Perceptron with {best_epoch} epoch and gaussian kernel with {gamma} : ', accuracy_test_gauss)

accuracy on train set for Perceptron with 41 epoch and gaussian kernel with gamma 2 :  0.992625
accuracy on test set for Perceptron with 41 epoch and gaussian kernel with 2 :  0.945


# V- Kernel Pegasos for SVM

In [148]:
class Kernel_Pegasos:
    
    def __init__(self, T, kernel):
        
        self.T = T                          # T is the number of rounds
        self.kernel = kernel
        self.lambda_param = None
        self.kernel_vectors = None
        self.kernel_vector_labels = None
        self.w_index = None
        self.param_kernel = None

    
    def train(self, X_train, Y_train, lambda_param, param_kernel):
        
        sample_size = X_train.shape[0]
        self.w_index = np.zeros(sample_size)
        self.kernel_vectors = X_train
        self.kernel_vector_labels = Y_train
        self.lambda_param = lambda_param    # lambda is the regularization coefficient
        self.param_kernel = param_kernel
        
        K = self.kernel(X_train, X_train, self.param_kernel)
        
        for t in range(self.T):
            
            lr = (1 / (self.lambda_param * (t + 1)))
            i = np.random.randint(0, sample_size)
            
            condition = Y_train[i] * lr * np.sum(self.w_index * Y_train * K[:, i])
            
            if (condition < 1):
                self.w_index[i] += 1
            

        
    def predict(self, X_test):
        
        K = self.kernel(X_test, self.kernel_vectors, self.param_kernel)
        Y_predicted = np.sign(np.sum(self.w_index * self.kernel_vector_labels * K, axis=1))

        return Y_predicted
    
    def evaluate(self, X_test, Y_test):
        
        Y_pred = self.predict(X_test)
        accuracy = np.mean(Y_pred == Y_test)
        
        return accuracy

In [164]:
def kernel_pegasos_tuning_parameter(algo, list_lr, list_kernel_param, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning):
    
    result = np.zeros((len(list_lr), len(list_kernel_param)))
    
    for l, e in enumerate(list_lr):
        for i, param in enumerate(list_kernel_param):
            algo.train(X_train_tuning, Y_train_tuning, e, param)
            result[l,i] = algo.evaluate(X_val_tuning, Y_val_tuning)
        print(f'step {l+1} over {len(list_lr)}')
            
    index_max = np.unravel_index(np.argmax(result), result.shape)
    best_lr = list_lr[index_max[0]]
    best_param = list_kernel_param[index_max[1]]
    
    return best_lr, best_param

## a) Kernel Pegasos with polynomial kernel

In [150]:
T = 30000  # we fix T big enough in hope to get convergence
list_degree = [i for i in range(1, 16)]
list_lambda = [0.001, 0.01, 0.1, 1]
best_lambda, best_degree = kernel_pegasos_tuning_parameter(Kernel_Pegasos(T,polynomial_kernel), list_lambda, list_degree, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best lambda : ', best_lambda ,'\n','best polynomial degree : ', best_degree)

0
1
2
3
4
5
best lambda :  0.1 
 best polynomial degree :  14


In [152]:
kernel_pegasos_pol = Kernel_Pegasos(T, polynomial_kernel)
kernel_pegasos_pol.train(X_train, Y_train, best_epoch, best_degree)
accuracy_train_pol = kernel_pegasos_pol.evaluate(X_train, Y_train)
accuracy_test_pol = kernel_pegasos_pol.evaluate(X_test, Y_test)
print(f'accuracy on train set for Pegasos with lambda = {best_lambda} and polynomial kernel of degree {best_degree} : ', accuracy_train_pol)
print(f'accuracy on test set for Pegasos with lambda = {best_lambda} and polynomial kernel of degree {best_degree} : ', accuracy_test_pol)

accuracy on train set for Pegasos with 0.1 lambda and polynomial kernel of degree 14 :  0.867875
accuracy on test set for Pegasos with 0.1 lambda and polynomial kernel of degree 14 :  0.8705


## b) Kernel Pegasos with gaussian kernel

In [153]:
T = 30000  # we fix T big enough in hope to get convergence
list_gamma = [0.1, 0.5, 1, 2, 5, 7, 10]
list_lambda = [0.001, 0.01, 0.1, 1]
best_lambda, best_gamma = kernel_pegasos_tuning_parameter(Kernel_Pegasos(T,gaussian_kernel), list_lambda, list_gamma, X_train_tuning, Y_train_tuning, X_val_tuning, Y_val_tuning)
print('best lambda : ', best_lambda ,'\n','best gamma : ', best_gamma)

0
1
2
3
4
5
best lambda :  0.001 
 best gamma :  0.1


In [155]:
kernel_pegasos_gauss = Kernel_Pegasos(T, gaussian_kernel)
kernel_pegasos_gauss.train(X_train, Y_train, best_lambda, best_gamma)
accuracy_train_gauss = kernel_pegasos_gauss.evaluate(X_train, Y_train)
accuracy_test_gauss = kernel_pegasos_gauss.evaluate(X_test, Y_test)
print(f'accuracy on train set for Pegasos with lambda = {best_lambda} and gaussian kernel of gamma {best_gamma} : ', accuracy_train_gauss)
print(f'accuracy on test set for Pegasos with lambda = {best_lambda} and gaussian kernel with gamma {best_gamma} : ', accuracy_test_gauss)

accuracy on train set for Pegasos with lambda = 0.001 and gaussian kernel of gamma 0.1 :  0.8675
accuracy on test set for Pegasos with lambda = 0.001 and gaussian kernel with gamma 0.1 :  0.863
